In [ ]:
#DEPENDENCIES
!pip install newsapi-python
!pip install --upgrade "ibm-watson>=4.2.1"
#!pip install tweepy

     |████████████████████████████████| 389kB 5.2MB/s 
     |████████████████████████████████| 204kB 31.8MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-5.1.0-cp36-none-any.whl size=375439 sha256=af93cc2e2833182c1e121aee0ef93d85d19f8a177aef03748021402af1f09b0a
  Stored in directory: /root/.cache/pip/wheels/49/6d/cf/1d91261b96363da78bf9b02699fd2262e6b5dad179500690c1
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.5.0-cp36-none-any.whl size=56069 sha256=2309e5ac88d7eb67f12e2d916924c1a1072bb62cd2b4f73e4852229b4b002a46
  Stored in directory: /root/.cache/pip/wheels/ec/36/df/b90cee121a9034824729a37e5e6c5e71ded9321ce221e492a7
Successfully built ibm-watson ibm-cloud-sdk-core


In [ ]:
#LOCAL DEPENDENCIES INSTALLER
import subprocess
import sys
 
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [ ]:
#IMPORT LIBRARIES
try:
  import tweepy
except:
  install("tweepy")
  import tweepy 
import json
import sys
#from google.colab import files
import datetime
from datetime import datetime
import time
 
try:
  from ibm_watson import PersonalityInsightsV3
  from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
except:
  subprocess.check_call([sys.executable, "--upgrade", "pip", "install", "ibm-watson>=4.2.1"])
  from ibm_watson import PersonalityInsightsV3
  from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
 
import requests
try:
  from newsapi import NewsApiClient
except:
  install("newsapi-python")

In [ ]:
class tweetFetch:
  consumer_key = "consumer_key"
  consumer_secret = "consumer_secret"
 
  access_key = "access_key"
  access_secret = "access_secret"
  def __init__(self,screen_name):
    self.screen_name=screen_name
  def getAllTweets(self,screen_name):
    auth=tweepy.OAuthHandler(self.consumer_key,self.consumer_secret)
    auth.set_access_token(self.access_key,self.access_secret)
    api=tweepy.API(auth)
    loop_check=0
    oldCount=0
    #create a list to store tweets
    allTweets=[]
    #make initial request for newest tweets
    newTweets=api.user_timeline(screen_name,count=200)
 
    allTweets.extend(newTweets)
 
    #save id of the 2nd last tweet
    oldest=allTweets[-1].id-1
 
    while len(newTweets) > 0 and loop_check!=1:
      print("...downloading new tweets from ",oldest)
 
      #200 new tweets
      newTweets=api.user_timeline(screen_name,count=200,max_id=oldest)
 
      #append to allTweets
      allTweets.extend(newTweets)
 
      #update the oldest cursor
      oldest=allTweets[-1].id-1
 
      print("...downloaded ",len(allTweets),"so far")
      if(oldCount==len(allTweets)):
        loop_check=1
 
 
    oldCount=len(allTweets)
    return allTweets
 
  def fetch_tweets(screen_names):
 
      # initialize the list to hold all tweets from all users
      alltweets=[]
 
      # get all tweets for each screen name
      for  screen_name in screen_names:
          alltweets.extend(getAllTweets(screen_name))
 
      return alltweets
 
  def store_tweets(self,alltweets,screen_name):
    tweet_list=[]
    fileN='%s_tweets.json'%screen_name
    contentItems=dict()
    contentItems['contentItems']=tweet_list
    print("ok",type(alltweets[1]))
    for tweet in alltweets:
      
      tweet_information=dict()
      tweet_information['content']=tweet.text.encode('utf-8')
      #tweet_information['content']=tweet.encode('utf-8')
      tweet_information['created']=int(tweet.created_at.strftime("%s"))
      tweet_information['id']=tweet.id_str
      if tweet.in_reply_to_screen_name != None:
        tweet_information['reply']=True
      else:
        tweet_information['reply']=False
        tweet_list.append(tweet_information)
      with open(fileN,'w',encoding="utf8")as f:
        text=json.dumps(contentItems,indent=2,default=str)
        f.write(text)
      f.close()

In [ ]:
class watsonAPI:
 
  def __init__(self,screen_name):
    self.screen_name=screen_name
  
 
  def valueCalculate(self,screen_name):
    apikey='API_KEY'
    url='API_URL'
    print("Processing Values....")
    auth=IAMAuthenticator(apikey)
    service = PersonalityInsightsV3(authenticator=auth, version='2017-10-13')
    service.set_service_url(url)
    fileP=open('%s_tweets.json'%screen_name)
    fileReadLine=fileP.read()
    tweetDict=json.loads(fileReadLine)
    #print(tweetDict[1]['content'])
 
    with open('%s_tweets.json'%screen_name,'r') as profile_json:
      profileJSON=service.profile(profile_json.read(),accept='application/json',content_type='application/json').get_result()
    profile_json.close()
    
    with open('%s_profile.json'%screen_name,'w')as j:
      text = json.dumps(profileJSON,indent=2,default=str)
      j.write(text)
    j.close()
    print("done")

In [ ]:
class newsAPI:
  
  
 
 
  def __init__(self,screen_name):
    self.access_token= 'ACCESS_TOKEN'
    self.screen_name=screen_name
    self.news= NewsApiClient(api_key=self.access_token)
 
  def getProfile(self,screen_name):
    fileP=open('%s_profile.json'%screen_name)
    fileReadLine=fileP.read()
    profile_dict=json.loads(fileReadLine)
    self.conservation=profile_dict['values'][0]['percentile']
    self.opennesstochange=profile_dict['values'][1]['percentile']
    self.hedonism=profile_dict['values'][2]['percentile']
    self.selfEnhancement=profile_dict['values'][3]['percentile']
    self.selfTranscendence=profile_dict['values'][4]['percentile']
  
  
  def newsSave(self,name,valueTag,query_headlines):
    with open("%s_%s.json"%(name,valueTag),'w') as n:
      text = json.dumps(query_headlines,indent=2,default=str)
      n.write(text)
    n.close()
 
  def newsGet(self,name):
    
    if(self.hedonism>0.2):
      query='entertainment'
      query_headlines=self.news.get_everything(q=query,sources='bbc-news,the-verge')
      print("Powered by newsapi.org"," hedonism")
      print(query_headlines)
      self.newsSave(name,'hedonism',query_headlines)
    if(self.selfTranscendence>0.2):
      query='environment'
      query_headlines=self.news.get_everything(q=query,sources='bbc-news,the-verge')
      print("Powered by newsapi.org"," selfTranscendence")
      print(query_headlines)
      self.newsSave(name,'selfTranscendence',query_headlines)
    if(self.selfEnhancement>0.2):
      query='career'
      query_headlines=self.news.get_everything(q=query,sources='bbc-news,the-verge')
      print("Powered by newsapi.org"," selfEnhancement")
      print(query_headlines)
      self.newsSave(name,'selfEnhancement',query_headlines)
    if(self.opennesstochange>0.2):
      query='future'
      query_headlines=self.news.get_everything(q=query,sources='bbc-news,the-verge',)
      print("Powered by newsapi.org"," opennesstochange")
      print(query_headlines)
      self.newsSave(name,'opennesstochange',query_headlines)
    if(self.conservation>0.2):
      query='social'
      query_headlines=self.news.get_everything(q=query,sources='bbc-news,the-verge',)
      print("Powered by newsapi.org"," conservation")
      print(query_headlines)
      self.newsSave(name,'conservation',query_headlines)

In [ ]:
screen_name="elonmusk"
 
 
tweetF=tweetFetch(screen_name)
watsonF=watsonAPI(screen_name)
newsF=newsAPI(screen_name)
 
tweetSave=dict()
tweetSave=tweetF.getAllTweets(screen_name)
tweetF.store_tweets(tweetSave,screen_name)
watsonF.valueCalculate(screen_name)
 
newsF.getProfile(screen_name)
newsF.newsGet(screen_name)

...downloading new tweets from  1347254119237439494
...downloaded  400 so far
...downloading new tweets from  1336006929739620353
...downloaded  600 so far
...downloading new tweets from  1327760574759878655
...downloaded  800 so far
...downloading new tweets from  1319609402878349311
...downloaded  1000 so far
...downloading new tweets from  1312175510244356095
...downloaded  1200 so far
...downloading new tweets from  1304627336470384639
...downloaded  1400 so far
...downloading new tweets from  1295837513987305473
...downloaded  1600 so far
...downloading new tweets from  1288889018512367615
...downloaded  1800 so far
...downloading new tweets from  1284282272007327743
...downloaded  1999 so far
...downloading new tweets from  1278760930596270084
...downloaded  2199 so far
...downloading new tweets from  1272655479420776448
...downloaded  2399 so far
...downloading new tweets from  1262901748777340933
...downloaded  2599 so far
...downloading new tweets from  1258315243844890623
...